# Exploration of Data.gov.sg's taxi and carpark availability API

Ref: https://beta.data.gov.sg/collections/85/view (carpark) ; https://beta.data.gov.sg/collections/352/view (taxi availability)

Data of interest: Realtime carpark and taxi availability.

## Notes:
- Carpark data only contains carpark ID without lat/lon location. A separate mapping file is required to facilitate location identification.
- For taxi availability, the coordinates and the total count are available.

In [2]:
from random import randint
from typing import Union, Dict
import requests
from PIL import Image

### Helper functions

In [14]:
def api_query(api_link: str,  agent_id: str) -> Union[Dict,None]:
    """Function which executes query via an api link using a provided agent_id as an identifier to avoid rejection of query request

    Args:
        api_link (str): API Link which requests is to be made
        agent_id (str): Id used for request header

    Returns:
        Dictioanry containing request content. None when exception are encountered.
    """
    req_headers = {"User-agent": agent_id}
    try:
        res = requests.get(api_link,
                            headers=req_headers,
                            timeout=5)
        # Raise if HTTPError occured
        res.raise_for_status()

        # Check the status code before extending the number of posts
        if res.status_code == 200:
            print(f"Request sucessful with {res.status_code}")
            the_json = res.json()
        return the_json

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

def query_taxi_availability(api_link: str = "https://api.data.gov.sg/v1//transport/taxi-availability", agent_id: str= "test_qzq"): 
    taxi_avail_response = api_query(api_link=api_link, agent_id=agent_id)
    if not taxi_avail_response:
        print("No response received")
        return {}
    else:
        #print(taxi_avail_response)
        taxi_avail_response_features = taxi_avail_response["features"][0]
        geo_coord = taxi_avail_response_features["geometry"]["coordinates"]
        print(f"Number of coordinates received {len(geo_coord)}")
        taxi_counts_dict = taxi_avail_response_features["properties"]
        print(geo_coord)
        print(taxi_counts_dict)
    return None



def query_carpark_availability(api_link: str = "https://api.data.gov.sg/v1/transport/carpark-availability", agent_id: str= "test_qzq"): 
    carpark_avail_response = api_query(api_link=api_link, agent_id=agent_id)

    carpark_dict = {}
    if not carpark_avail_response:
        print("No response received")
        
    else:
        print(carpark_avail_response)
        carpark_data_list = carpark_avail_response["items"][0]["carpark_data"]
        for carpark_info_dict in carpark_data_list:
            carpark_number = carpark_info_dict.pop("carpark_number")
            carpark_capacity_info = carpark_info_dict["carpark_info"][0]
            if carpark_number not in carpark_dict:
                carpark_dict[carpark_number] = carpark_capacity_info
            else:
                carpark_dict[carpark_number].update(carpark_capacity_info)
    return carpark_dict

### Function call to explore response outputs

In [15]:
query_taxi_availability()

Request sucessful with 200
Number of coordinates received 2079
[[103.62226, 1.28117], [103.62465, 1.30338], [103.62548, 1.28945], [103.62684, 1.29421], [103.62831, 1.2942], [103.6298754, 1.28930351666667], [103.6314, 1.3081], [103.63229, 1.30719], [103.63589, 1.3199], [103.63629, 1.33051], [103.63759, 1.30026], [103.63766, 1.30037], [103.63768, 1.3005], [103.64068, 1.32948], [103.64116, 1.30075], [103.64316, 1.3264], [103.6432, 1.33405], [103.64394, 1.3289], [103.65571, 1.32676], [103.65778, 1.30622], [103.66072, 1.3104], [103.66172, 1.31658], [103.66608, 1.30944], [103.66663, 1.31241], [103.66678, 1.31143], [103.66718, 1.32741], [103.6682845, 1.31195316666667], [103.6722, 1.33006], [103.67385, 1.31227], [103.67665, 1.31933], [103.6767, 1.31869], [103.6786, 1.32734], [103.6788, 1.3111], [103.67889, 1.31396], [103.67894, 1.31624], [103.67925, 1.31512], [103.6794905, 1.3416705], [103.68202, 1.3396], [103.684, 1.32712], [103.68453, 1.35516], [103.68569, 1.34223], [103.68572485, 1.32987985

In [7]:
query_carpark_availability()

Request sucessful with 200
{'items': [{'timestamp': '2023-11-14T18:03:27+08:00', 'carpark_data': [{'carpark_info': [{'total_lots': '105', 'lot_type': 'C', 'lots_available': '0'}], 'carpark_number': 'HE12', 'update_datetime': '2023-11-14T18:02:10'}, {'carpark_info': [{'total_lots': '583', 'lot_type': 'C', 'lots_available': '185'}], 'carpark_number': 'HLM', 'update_datetime': '2023-11-14T18:03:06'}, {'carpark_info': [{'total_lots': '329', 'lot_type': 'C', 'lots_available': '201'}], 'carpark_number': 'RHM', 'update_datetime': '2023-11-14T18:02:10'}, {'carpark_info': [{'total_lots': '97', 'lot_type': 'C', 'lots_available': '31'}], 'carpark_number': 'BM29', 'update_datetime': '2023-11-14T18:02:00'}, {'carpark_info': [{'total_lots': '96', 'lot_type': 'C', 'lots_available': '25'}], 'carpark_number': 'Q81', 'update_datetime': '2023-11-14T18:02:52'}, {'carpark_info': [{'total_lots': '176', 'lot_type': 'C', 'lots_available': '41'}], 'carpark_number': 'C20', 'update_datetime': '2023-11-14T18:03:0

{'HE12': {'total_lots': '105', 'lot_type': 'C', 'lots_available': '0'},
 'HLM': {'total_lots': '583', 'lot_type': 'C', 'lots_available': '185'},
 'RHM': {'total_lots': '329', 'lot_type': 'C', 'lots_available': '201'},
 'BM29': {'total_lots': '97', 'lot_type': 'C', 'lots_available': '31'},
 'Q81': {'total_lots': '96', 'lot_type': 'C', 'lots_available': '25'},
 'C20': {'total_lots': '176', 'lot_type': 'C', 'lots_available': '41'},
 'FR3M': {'total_lots': '228', 'lot_type': 'C', 'lots_available': '167'},
 'C32': {'total_lots': '289', 'lot_type': 'C', 'lots_available': '230'},
 'C6': {'total_lots': '332', 'lot_type': 'C', 'lots_available': '141'},
 'TG2': {'total_lots': '273', 'lot_type': 'C', 'lots_available': '125'},
 'BP1': {'total_lots': '577', 'lot_type': 'C', 'lots_available': '389'},
 'TG1': {'total_lots': '133', 'lot_type': 'C', 'lots_available': '87'},
 'TGM2': {'total_lots': '189', 'lot_type': 'C', 'lots_available': '144'},
 'TE14': {'total_lots': '134', 'lot_type': 'C', 'lots_av